In [1]:
#Results were produced in stints. This is the number of the last stint.
run = 4

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from scipy import stats
from scipy import optimize
import joblib

#folder for saving results
filepath = ".../Resultate_final/Put/IS/250/Put_VaR_ES_IS_250_sim_saved/"

In [3]:
#Market and option parameters as in section 4.1 of 'Assessing Asset-Liability Risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
s_0 = 100
r = 0.01
mu = 0.05
sigma = 0.2
tau = 1/52
T = 1/3
K = 100

#Confidence levels and parameters for Value-at-Risk, Expected Shortfall
alpha_VaR = 0.995
alpha_ES = 0.99

In [4]:
#Sizes for training set, validation set, test set, and set size for Monte Carlo estimation of the risk measures
M_1 = 1500000
M_2 = 500000
M_3 = 500000
M_MC = 500000
#size of the set of data points used to calculate an IS density
M_IS = 250000
#quantile for which the IS density will be computed
alpha_IS = 0.995

In [5]:
#Function for calculating simulated values of S_tau and simulated payoffs P_T from simulations of standard normal random variables
def data_gen(Z,V):
    #simulate S_tau under P
    S_tau = s_0 * np.exp( (mu-0.5*sigma**2)*tau + sigma*np.sqrt(tau)*Z)
    #simulate S_T given S_tau under Q
    S_T = S_tau * np.exp( (r-0.5*sigma**2)*(T-tau) + sigma*np.sqrt(T-tau)*V)
    #calculate corresponding discounted payoffs
    P_T = np.exp(-r*(T-tau)) * np.maximum(K-S_T,0)
    return S_tau, P_T

#The density function of Z
def f(y):
    return stats.norm.pdf(y, loc=0, scale=1)

#The density function of Z_\theta (note that \theta is replaced by x; this is needed for the least-squares solver to work)
def f_theta(x, y):
    return stats.norm.pdf(y, loc=x[0], scale=x[1])

#This function describes the approximation of the expression inside the sum of m_2(theta)
def g_q_alpha_hat_reweighted(x,L,q_alpha_hat):
    return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)

#bounds for the IS density parameters (for the mean parameter no bound is necessary, the standard deviation however needs to be non-negative)
bnds_lower = np.array([-np.inf,0])
bnds_upper = np.array([np.inf,np.inf])

In [6]:
for j in range(59):
    #Generating realisations of standard normal random variables
    Z_train = np.random.normal(loc=0, scale=1, size=M_1)
    V_train = np.random.normal(loc=0, scale=1, size=M_1)
    Z_val = np.random.normal(loc=0, scale=1, size=M_2)
    V_val = np.random.normal(loc=0, scale=1, size=M_2)
    Z_test = np.random.normal(loc=0, scale=1, size=M_3)
    V_test = np.random.normal(loc=0, scale=1, size=M_3)
    Z_MC = np.random.normal(loc=0, scale=1, size=M_MC)
    V_MC = np.random.normal(loc=0, scale=1, size=M_MC)

    Z_IS = np.random.normal(loc=0, scale=1, size=M_IS)
    V_IS = np.random.normal(loc=0, scale=1, size=M_IS)
    S_tau_IS, P_T_IS = data_gen(Z=Z_IS, V=V_IS)


    #define and compile neural network model, setup as in section 4.1 of 'Assessing Asset-Liability Risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
    bi_IS = np.log( np.sum(P_T_IS)/len(P_T_IS))
    model_IS = tf.keras.models.Sequential([
        tf.keras.layers.BatchNormalization(input_shape=(1,)),
        tf.keras.layers.Dense(5, activation='tanh'),
        tf.keras.layers.Dense(1, activation='exponential', bias_initializer=tf.keras.initializers.Constant(value=bi_IS))])
    model_IS.compile(loss='mse', optimizer='adam', metrics=['mse'])
    #train the neural network
    b=model_IS.fit(x=S_tau_IS, y=P_T_IS, epochs=40, batch_size=10000, verbose=0)

    #computation of \theta^*_{NN}
    L_IS = np.column_stack((Z_IS, model_IS.predict(S_tau_IS)[:,0]))
    L_IS = L_IS[L_IS[:,-1].argsort()[::-1]]
    q_alpha_IS_hat = L_IS[int(M_IS*(1-alpha_IS)-1), -1]
    print('q_alpha_IS_hat_NN:', q_alpha_IS_hat)
    
    IS_NN = optimize.least_squares(g_q_alpha_hat_reweighted, x0=np.array([0,1]), bounds=(bnds_lower,bnds_upper), args=(L_IS, q_alpha_IS_hat)).x
    print('IS_NN:', IS_NN)
    #a runtime warning may occur here but in all trial runs this was not of concern as the optimization algorithm still executed successfully and the result was meaningful

    #calculating DT(Z,\theta^*_{NN})
    Z_train_NN = Z_train*IS_NN[1] + IS_NN[0]
    Z_val_NN = Z_val*IS_NN[1] + IS_NN[0]
    Z_test_NN = Z_test*IS_NN[1] + IS_NN[0]
    Z_MC_NN = Z_MC*IS_NN[1] + IS_NN[0]
    #calculating the risk factors under the IS distribution and corresponding option prices
    S_tau_train_NN, P_T_train_NN = data_gen(Z=Z_train_NN, V=V_train)
    S_tau_val_NN, P_T_val_NN = data_gen(Z=Z_val_NN, V=V_val)
    S_tau_test_NN, P_T_test_NN = data_gen(Z=Z_test_NN, V=V_test)
    S_tau_MC_NN, P_T_MC_NN = data_gen(Z=Z_MC_NN, V=V_MC)

    #define and compile neural network model, setup as in section 4.1 of 'Assessing Asset-Liability Risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
    bi = np.log( np.sum(P_T_train_NN)/len(P_T_train_NN))
    model = tf.keras.models.Sequential([
        tf.keras.layers.BatchNormalization(input_shape=(1,)),
        tf.keras.layers.Dense(5, activation='tanh'),
        tf.keras.layers.Dense(1, activation='exponential', bias_initializer=tf.keras.initializers.Constant(value=bi))])
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    #train the neural network
    hist = model.fit(x=S_tau_train_NN, y=P_T_train_NN, epochs=40, batch_size=10000, validation_data=(S_tau_val_NN,P_T_val_NN), verbose=0)

    #Calculating the parts of the test set that fall into B_1 and B_2
    s_40 = s_0 * np.exp( (mu-0.5*sigma**2)*tau + sigma*np.sqrt(tau)*stats.norm.ppf(0.4, loc=0, scale=1))
    s_70 = s_0 * np.exp( (mu-0.5*sigma**2)*tau + sigma*np.sqrt(tau)*stats.norm.ppf(0.7, loc=0, scale=1))
    B_1_NN = S_tau_test_NN < s_40
    B_2_NN = S_tau_test_NN > s_70

    #computation of the metrics (a), (b), (c) with B_1 and (c) with B_2 for the neural network
    P_T_pred_NN = model.predict(S_tau_test_NN)[:,0]
    mse_train_NN = hist.history['mse'][-1]
    mse_val_NN = hist.history['val_mse'][-1]
    mc_tmp = P_T_pred_NN - P_T_test_NN
    metric_a_NN = np.sum(mc_tmp)/len(P_T_test_NN)
    metric_b_NN = np.sum((mc_tmp)*P_T_pred_NN)/len(P_T_test_NN)
    metric_c_B_1_NN = np.sum(mc_tmp[B_1_NN])/len(P_T_test_NN)
    metric_c_B_2_NN = np.sum(mc_tmp[B_2_NN])/len(P_T_test_NN)

    #computation of Value-at-Risk and Expected Shortfall using the neural network
    P_T_pred_MC_NN = model.predict(S_tau_MC_NN)[:,0]
    MC = np.column_stack((Z_MC_NN, P_T_pred_MC_NN))
    MC_sort = MC[MC[:,-1].argsort()[::-1]]
    w = f(MC_sort[:,0])/(M_MC*f_theta(y=MC_sort[:,0], x=IS_NN))

    j_VaR = 0
    w_sum_tmp = 0
    while (w_sum_tmp <= (1-alpha_VaR) and j_VaR<M_MC):
        w_sum_tmp += w[j_VaR]
        j_VaR += 1
    VaR_hat_NN = MC_sort[j_VaR-1,-1]
    print('VaR_hat_NN:',VaR_hat_NN)
    
    j_ES = 0
    w_sum_tmp = 0
    while (w_sum_tmp <= (1-alpha_ES) and j_ES<M_MC):
        w_sum_tmp += w[j_ES]
        j_ES += 1
    ES_hat_NN = 1/(1-alpha_ES) * np.sum(w[0:j_ES-1]*MC_sort[0:j_ES-1,1]) + ( 1 - (1 / (1-alpha_ES)) * np.sum(w[0:j_ES-1]) )*MC_sort[j_ES,1]
    print('ES_hat_NN:',ES_hat_NN)


    #define and train random forest with the same hyperparameter as in Put_VaR_ES_IS_250.ipynb
    rfr_IS = RandomForestRegressor(n_estimators=160, criterion='squared_error', min_samples_leaf=250, bootstrap=True, verbose=0, n_jobs=-1)
    rfr_IS.fit(X=S_tau_IS.reshape(-1,1), y=P_T_IS)

    #computation of \theta^*_{RF}
    L_IS = np.column_stack((Z_IS, rfr_IS.predict(S_tau_IS.reshape(-1,1))))
    L_IS = L_IS[L_IS[:,-1].argsort()[::-1]]
    q_alpha_IS_hat = L_IS[int(M_IS*(1-alpha_IS)-1), -1]
    print('q_alpha_IS_hat_RF:', q_alpha_IS_hat)
    
    IS_RF = optimize.least_squares(g_q_alpha_hat_reweighted, x0=np.array([0,1]), bounds=(bnds_lower,bnds_upper), args=(L_IS, q_alpha_IS_hat)).x
    print('IS_RF:', IS_RF)
    #a runtime warning may occur here but in all trial runs this was not of concern as the optimization algorithm still executed successfully and the result was meaningful

    #calculating DT(Z,\theta^*_{RF})
    Z_train_RF = Z_train*IS_RF[1] + IS_RF[0]
    Z_val_RF = Z_val*IS_RF[1] + IS_RF[0]
    Z_test_RF = Z_test*IS_RF[1] + IS_RF[0]
    Z_MC_RF = Z_MC*IS_RF[1] + IS_RF[0]
    #calculating the risk factors under the IS distribution and corresponding option prices
    S_tau_train_RF, P_T_train_RF = data_gen(Z=Z_train_RF, V=V_train)
    S_tau_val_RF, P_T_val_RF = data_gen(Z=Z_val_RF, V=V_val)
    S_tau_test_RF, P_T_test_RF = data_gen(Z=Z_test_RF, V=V_test)
    S_tau_MC_RF, P_T_MC_RF = data_gen(Z=Z_MC_RF, V=V_MC)
    
    #hypyerparameter tuning with orientation around the value found in Put_VaR_ES_IS_250.ipynb
    min_samples_leaf_list = [5000,5200,5400]
    opt_param = 0
    opt_score = np.inf

    for min_samples_leaf in min_samples_leaf_list:
        rfr_tuning = RandomForestRegressor(n_estimators=160, min_samples_leaf=min_samples_leaf, bootstrap=True, criterion='squared_error', verbose=0, n_jobs=-1)
        rfr_tuning.fit(X=S_tau_train_RF.reshape(-1,1), y=P_T_train_RF)
        score = mean_squared_error(y_true=P_T_val_RF, y_pred=rfr_tuning.predict(S_tau_val_RF.reshape(-1,1)))
        if score < opt_score:
            opt_param = min_samples_leaf
            opt_score = score

    #definition and training of a random forest with hyperparameters found through grid search
    rfr = RandomForestRegressor(n_estimators=400, criterion='squared_error', min_samples_leaf=int(opt_param), bootstrap=True, verbose=0, warm_start=True, n_jobs=-1)
    rfr.fit(X=S_tau_train_RF.reshape(-1,1), y=P_T_train_RF)

    #Calculating the parts of the test set that fall into B_1 and B_2
    s_40 = s_0 * np.exp( (mu-0.5*sigma**2)*tau + sigma*np.sqrt(tau)*stats.norm.ppf(0.4, loc=0, scale=1))
    s_70 = s_0 * np.exp( (mu-0.5*sigma**2)*tau + sigma*np.sqrt(tau)*stats.norm.ppf(0.7, loc=0, scale=1))
    B_1_RF = S_tau_test_RF < s_40
    B_2_RF = S_tau_test_RF > s_70

    #computation of the metrics (a), (b), (c) with B_1 and (c) with B_2 and training/validation MSE for the random forest
    mse_train_RF = mean_squared_error(y_pred=rfr.predict(S_tau_train_RF.reshape(-1,1)),y_true=P_T_train_RF)
    P_T_pred_RF = rfr.predict(S_tau_test_RF.reshape(-1,1))
    mse_val_RF = mean_squared_error(y_pred=P_T_pred_RF,y_true=P_T_test_RF)
    mc_tmp = P_T_pred_RF - P_T_test_RF
    metric_a_RF = np.sum(mc_tmp)/len(P_T_test_RF)
    metric_b_RF = np.sum((mc_tmp)*P_T_pred_RF)/len(P_T_test_RF)
    metric_c_B_1_RF = np.sum(mc_tmp[B_1_RF])/len(P_T_test_RF)
    metric_c_B_2_RF = np.sum(mc_tmp[B_2_RF])/len(P_T_test_RF)

    #computation of Value-at-Risk and Expected Shortfall using the random forest
    P_T_pred_MC_RF = rfr.predict(S_tau_MC_RF.reshape(-1,1))
    MC = np.column_stack((Z_MC_RF, P_T_pred_MC_RF))
    MC_sort = MC[MC[:,-1].argsort()[::-1]]
    w = f(MC_sort[:,0])/(M_MC*f_theta(y=MC_sort[:,0], x=IS_RF))

    j_VaR = 0
    w_sum_tmp = 0
    while (w_sum_tmp <= (1-alpha_VaR) and j_VaR<M_MC):
        w_sum_tmp += w[j_VaR]
        j_VaR += 1
    VaR_hat_RF = MC_sort[j_VaR-1,-1]
    print('VaR_hat_RF:',VaR_hat_RF)
    
    j_ES = 0
    w_sum_tmp = 0
    while (w_sum_tmp <= (1-alpha_ES) and j_ES<M_MC):
        w_sum_tmp += w[j_ES]
        j_ES += 1
    ES_hat_RF = 1/(1-alpha_ES) * np.sum(w[0:j_ES-1]*MC_sort[0:j_ES-1,1]) + ( 1 - (1 / (1-alpha_ES)) * np.sum(w[0:j_ES-1]) )*MC_sort[j_ES,1]
    print('ES_hat_RF:',ES_hat_RF)

    #save results for further evaluation
    output = np.array([[mse_train_NN,mse_val_NN,metric_a_NN,metric_b_NN,metric_c_B_1_NN,metric_c_B_2_NN,IS_NN[0],IS_NN[1],VaR_hat_NN,ES_hat_NN],
                      [mse_train_RF,mse_val_RF,metric_a_RF,metric_b_RF,metric_c_B_1_RF,metric_c_B_2_RF,IS_RF[0],IS_RF[1],VaR_hat_RF,ES_hat_RF]])

    joblib.dump(output,filepath+'output_'+str(run)+'_'+str(j)+'.joblib')
    #prints just for checking while the notebook is running
    print(j)

2021-11-07 14:49:34.187816: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-07 14:49:36.331633: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30988 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:89:00.0, compute capability: 7.0
2021-11-07 14:49:36.333267: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30988 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8a:00.0, compute capability: 7.0
2021-11-07 14:49:36.334831: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:

q_alpha_IS_hat_NN: 7.7089524269104
IS_NN: [-2.83236024  0.43039434]
VaR_hat_NN: 8.362968444824219
ES_hat_NN: 8.517113272326524
q_alpha_IS_hat_RF: 8.54971778998601
IS_RF: [-2.70870018  0.45201889]
VaR_hat_RF: 8.359840853294342
ES_hat_RF: 8.516423097441002
0
q_alpha_IS_hat_NN: 7.5228142738342285


/scratch/slurm_tmpdir/job_20161366/ipykernel_2977501/3406871675.py:15: RuntimeWarning: divide by zero encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20161366/ipykernel_2977501/3406871675.py:15: RuntimeWarning: overflow encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20161366/ipykernel_2977501/3406871675.py:15: RuntimeWarning: invalid value encountered in multiply
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)


IS_NN: [-2.83706132  0.41041628]
VaR_hat_NN: 8.389033317565918
ES_hat_NN: 8.54531864264372
q_alpha_IS_hat_RF: 8.340464946880392


/scratch/slurm_tmpdir/job_20161366/ipykernel_2977501/3406871675.py:15: RuntimeWarning: divide by zero encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20161366/ipykernel_2977501/3406871675.py:15: RuntimeWarning: overflow encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20161366/ipykernel_2977501/3406871675.py:15: RuntimeWarning: invalid value encountered in multiply
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)


...

IS_RF: [-2.75385015  0.41583885]
VaR_hat_RF: 8.345923059781159
ES_hat_RF: 8.501749657679547
57
q_alpha_IS_hat_NN: 6.482161521911621


/scratch/slurm_tmpdir/job_20161366/ipykernel_2977501/3406871675.py:15: RuntimeWarning: divide by zero encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20161366/ipykernel_2977501/3406871675.py:15: RuntimeWarning: overflow encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20161366/ipykernel_2977501/3406871675.py:15: RuntimeWarning: invalid value encountered in multiply
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)


IS_NN: [-2.71090603  0.18636336]
VaR_hat_NN: 8.330083847045898
ES_hat_NN: 8.500382718006069
q_alpha_IS_hat_RF: 8.923974440332056


/scratch/slurm_tmpdir/job_20161366/ipykernel_2977501/3406871675.py:15: RuntimeWarning: divide by zero encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20161366/ipykernel_2977501/3406871675.py:15: RuntimeWarning: overflow encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20161366/ipykernel_2977501/3406871675.py:15: RuntimeWarning: invalid value encountered in multiply
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)


IS_RF: [-2.69134054  0.38691762]
VaR_hat_RF: 8.32093691231588
ES_hat_RF: 8.500113423927013
58
